# 3 Exploring Big Data (Optional Technical Task)

This is the Third(optional) taksk of the Data@ANZ Virtual Experiance program.  
Tasks : 
1. Create Spark application using both the RDD and Dataframe API through different classes or functions.

2. Using each API, perform the following transformation steps using the synthetic transaction file as input referenced as an input argument to your program. Output the results to a local file.

Project only the records where status=authorized AND card_present_flag=0
Split the long_lat and merchant_long_lat fields into long, lat and merch_long, merch_lat fields
Output the data as a CSV file
3. Time each operation and report on the differences in execution time between each API. Print the output using stdout in your program. 

You are free to develop your application using any Spark first class programming language, preferably Scala or Java however.

While designing your program and code, make sure your program is robust, maintainable and scalable. 

4. For a challenge!

Create a program to read each line in the file and perform the same steps using a Spark Streaming (or Structured Streaming) application, using the console as output.

Submit your report (in pdf format).

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/spark_data/')
print(os.getcwd())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/spark_data


In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

from pyspark.sql import SQLContext 
sql = SQLContext(sc)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [3]:
# from google.colab import files
# uploaded = files.upload()



In [4]:
!ls

BIGDATA_Content_Task.xlsx  spark-2.3.1-bin-hadoop2.7.tgz
spark-2.3.1-bin-hadoop2.7  spark-2.3.1-bin-hadoop2.7.tgz.1


## Using DataFrame API

In [5]:
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, TimestampType, BooleanType, IntegerType
import pandas as pd

stk = [
('status',StringType()), 
('card_present_flag',StringType()),
('bpay_biller_code',StringType()),
('account',StringType()),
('currency',StringType()), 
('long_lat',StringType()), 
('txn_description',StringType()), 
('merchant_id',StringType()),
('merchant_code',StringType()), 
('first_name',StringType()), 
('balance',DoubleType()), 
('date',TimestampType()), 
('gender',StringType()), 
('age',IntegerType()),
('merchant_suburb',StringType()), 
('merchant_state',StringType()), 
('extraction',StringType()), 
('amount',DoubleType()),
('transaction_id',StringType()), 
('country',StringType()), 
('customer_id',StringType()), 
('merchant_long_lat',StringType()),
('movement',StringType())]

schema = StructType([StructField(x[0], x[1],True) for x in stk])
data = pd.read_excel('BIGDATA_Content_Task.xlsx')
df = spark.createDataFrame(data, schema = schema) 
df.show(5)


+----------+-----------------+----------------+--------------+--------+-------------+---------------+--------------------+-------------+----------+-------+-------------------+------+---+---------------+--------------+--------------------+------+--------------------+---------+--------------+-----------------+--------+
|    status|card_present_flag|bpay_biller_code|       account|currency|     long_lat|txn_description|         merchant_id|merchant_code|first_name|balance|               date|gender|age|merchant_suburb|merchant_state|          extraction|amount|      transaction_id|  country|   customer_id|merchant_long_lat|movement|
+----------+-----------------+----------------+--------------+--------+-------------+---------------+--------------------+-------------+----------+-------+-------------------+------+---+---------------+--------------+--------------------+------+--------------------+---------+--------------+-----------------+--------+
|authorized|              1.0|             

In [6]:
from pyspark.sql.functions import col, substring, split


### records with status = authorized and card_present_flag = 0

In [7]:
auth_card = df.filter((col('status')=='authorized') &  (col('card_present_flag')=='0.0'))
auth_card.show(5)
auth_card.count()

+----------+-----------------+----------------+--------------+--------+-------------+---------------+--------------------+-------------+----------+--------+-------------------+------+---+---------------+--------------+--------------------+------+--------------------+---------+--------------+-----------------+--------+
|    status|card_present_flag|bpay_biller_code|       account|currency|     long_lat|txn_description|         merchant_id|merchant_code|first_name| balance|               date|gender|age|merchant_suburb|merchant_state|          extraction|amount|      transaction_id|  country|   customer_id|merchant_long_lat|movement|
+----------+-----------------+----------------+--------------+--------+-------------+---------------+--------------------+-------------+----------+--------+-------------------+------+---+---------------+--------------+--------------------+------+--------------------+---------+--------------+-----------------+--------+
|authorized|              0.0|          

1523

### splitting the merchant_long_lat and long_lat in to seperate columns

In [8]:
help(split)

Help on function split in module pyspark.sql.functions:

split(str, pattern)
    Splits str around pattern (pattern is a regular expression).
    
    .. note:: pattern is a string represent the regular expression.
    
    >>> df = spark.createDataFrame([('ab12cd',)], ['s',])
    >>> df.select(split(df.s, '[0-9]+').alias('s')).collect()
    [Row(s=['ab', 'cd'])]
    
    .. versionadded:: 1.5



In [9]:
auth_card = auth_card.withColumn('long',split(auth_card['long_lat'],' ').getItem(0))
auth_card = auth_card.withColumn('lat',split(auth_card['long_lat'],' ').getItem(1))
auth_card = auth_card.withColumn('merch_long',split(auth_card['merchant_long_lat'],' ').getItem(0))
auth_card = auth_card.withColumn('merch_lat',split(auth_card['merchant_long_lat'],' ').getItem(1))

auth_card.show(5)

+----------+-----------------+----------------+--------------+--------+-------------+---------------+--------------------+-------------+----------+--------+-------------------+------+---+---------------+--------------+--------------------+------+--------------------+---------+--------------+-----------------+--------+------+------+----------+---------+
|    status|card_present_flag|bpay_biller_code|       account|currency|     long_lat|txn_description|         merchant_id|merchant_code|first_name| balance|               date|gender|age|merchant_suburb|merchant_state|          extraction|amount|      transaction_id|  country|   customer_id|merchant_long_lat|movement|  long|   lat|merch_long|merch_lat|
+----------+-----------------+----------------+--------------+--------+-------------+---------------+--------------------+-------------+----------+--------+-------------------+------+---+---------------+--------------+--------------------+------+--------------------+---------+-------------

In [16]:
# auth_card.write.format("csv").save("auth_card_dataframe.csv")
# files.download("auth_card_dataframe.csv")

auth_card_pd = auth_card.toPandas()
auth_card_pd.to_csv('auth_card_dataframe.csv')

In [19]:
!ls

auth_card_dataframe.csv    spark-2.3.1-bin-hadoop2.7.tgz
BIGDATA_Content_Task.xlsx  spark-2.3.1-bin-hadoop2.7.tgz.1
spark-2.3.1-bin-hadoop2.7
